In [1]:
import requests

def get_google_apis(preferred=False, name=None):
    # Base URL for the Google APIs Discovery Service
    url = "https://discovery.googleapis.com/discovery/v1/apis"
    
    # Parameters for the request
    params = {
        'preferred': str(preferred).lower()  # Convert boolean to string
    }
    
    if name:
        params['name'] = name  # Add name parameter if provided

    # Make the GET request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        return response.json()  # Return the JSON response
    else:
        response.raise_for_status()  # Raise an error for bad responses

# Example usage


In [4]:
from prettytable import PrettyTable
def print_apis_table(apis):
    # Create a PrettyTable object
    table = PrettyTable()
    table.field_names = ["ID", "Name", "Version", "Title", "Description", "Preferred"]

    # Populate the table with API data
    for api in apis.get('items', []):
        table.add_row([
            api.get('id'),
            api.get('name'),
            api.get('version'),
            api.get('title'),
            api.get('description'),
            api.get('preferred')
        ])

    # Print the table
    print(table)

In [5]:

try:
    apis = get_google_apis(preferred=True)
    print_apis_table(apis)  # Print the list of APIs
except Exception as e:
    print(f"An error occurred: {e}")

+-------------------------------------+----------------------------------+-----------------+----------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+
|                  ID                 |               Name               |    

In [10]:
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build

def authenticate_google_business_api():
    credentials = Credentials.from_authorized_user_file(
        '/home/vikas/Code/seoclientmanager/google_secrets.json', 
        scopes=['https://www.googleapis.com/auth/business.manage']
    )
    return build('mybusiness', 'v4', credentials=credentials)

In [12]:
from google.oauth2 import service_account
from googleapiclient.discovery import build

def authenticate_with_service_account(service_account_file):
    # Scopes for Business Profile API
    SCOPES = ['https://www.googleapis.com/auth/business.manage']
    
    # Load credentials from service account JSON file
    credentials = service_account.Credentials.from_service_account_file(
        service_account_file, 
        scopes=SCOPES
    )
    
    # Impersonation (if needed)
    credentials = credentials.with_subject('your_google_workspace_admin_email@domain.com')
    
    # Build Business Profile API service
    service = build('mybusiness', 'v4', credentials=credentials)
    
    return service 

In [13]:
def get_business_categories(service, country_code='US', language_code='en'):
    categories = []
    page_token = None

    while True:
        # Retrieve categories with pagination
        results = service.categories().list(
            regionCode=country_code,
            languageCode=language_code,
            pageSize=100,
            pageToken=page_token
        ).execute()

        # Extend categories list
        categories.extend(results.get('categories', []))

        # Check for more pages
        page_token = results.get('nextPageToken')
        if not page_token:
            break

    return categories


In [15]:

    # Implement retry or fallback mechanism

Authentication failed: name: mybusiness  version: v4


In [11]:

service = authenticate_google_business_api()
categories = get_business_categories(service)
print(categories)

ValueError: Authorized user info was not in the expected format, missing fields refresh_token, client_id, client_secret.

In [19]:
import googleapiclient.discovery

def get_all_categories(language_code, region_code=None):
    """Gets all business categories for the provided language.

    Args:
        language_code: The BCP 47 code of language.
        region_code: (Optional) The ISO 3166-1 alpha-2 country code.

    Returns:
        A list of all categories.
    """
    try:
        service = authenticate_with_service_account('neuralamiai-d25fb0dbeae0.json')
    except Exception as e:
        print(f"Authentication failed: {e}")
    #service = googleapiclient.discovery.build('mybusinessbusinessinformation', 'v1')

    all_categories = []
    next_page_token = None

    while True:
        request_body = {
            'languageCode': language_code,
            'pageSize': 100  # Get maximum categories per page
        }
        if region_code:
            request_body['regionCode'] = region_code
        if next_page_token:
            request_body['pageToken'] = next_page_token

        response = service.categories().list(**request_body).execute()
        all_categories.extend(response['categories'])

        next_page_token = response.get('nextPageToken')
        if not next_page_token:
            break

    return all_categories

# Example usage
all_categories = get_all_categories(language_code='en-US')

print(len(all_categories))  # Print the total number of categories
print(all_categories)  # Print the list of categories

Authentication failed: name: mybusiness  version: v4


UnboundLocalError: local variable 'service' referenced before assignment